In [1]:
# Import necessary libraries
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
from typing import List, Dict
import time

# Load model and tokenizer with optimization
model_name = "ibm-granite/granite-3.2-2b-instruct"

# Use a simpler model for faster response or keep the same with optimizations
# For faster response, consider using a smaller model like:
# model_name = "microsoft/DialoGPT-medium"  # Alternative smaller model

# Load with optimizations
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
    low_cpu_mem_usage=True
)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Model optimization for inference
model.eval()
if torch.cuda.is_available():
    model = model.to('cuda')

# Predefined conditions and recommendations for common symptoms
PREDEFINED_RESPONSES = {
    "fever": "Fever can be caused by various conditions including infections (viral or bacterial), inflammatory conditions, or other medical issues. Common recommendations include rest, hydration, and over-the-counter fever reducers like acetaminophen or ibuprofen. However, persistent fever requires medical evaluation.",
    "headache": "Headaches can be tension-type, migraine, or related to other conditions. Rest in a quiet, dark room, hydration, and over-the-counter pain relievers may help. Seek medical attention for severe, sudden, or persistent headaches.",
    "cough": "Cough can be due to respiratory infections, allergies, or other conditions. Hydration, humidifiers, and cough suppressants may provide relief. Consult a doctor if cough persists or is accompanied by fever or difficulty breathing.",
}

# Cache for recent queries
response_cache = {}
CACHE_SIZE = 100  # Keep last 100 queries in cache

def generate_response_optimized(prompt: str, max_length: int = 600) -> str:
    """Optimized response generation with caching and length control"""
    # Check cache first
    if prompt in response_cache:
        return response_cache[prompt]

    # Tokenize with truncation
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    # Move to device if CUDA available
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate with optimized parameters
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            early_stopping=True
        )

    # Decode and clean response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()

    # Clean up response
    response = re.sub(r'\s+', ' ', response)  # Remove extra whitespace
    response = re.sub(r'([.!?]) +', r'\1\n\n', response)  # Add paragraph breaks

    # Cache the response
    if len(response_cache) >= CACHE_SIZE:
        # Remove oldest item if cache is full
        response_cache.pop(next(iter(response_cache)))
    response_cache[prompt] = response

    return response

def disease_prediction(symptoms: str) -> str:
    """Optimized disease prediction with predefined responses for common symptoms"""
    # Check for common symptoms first
    symptoms_lower = symptoms.lower()
    common_symptoms = []

    for symptom in PREDEFINED_RESPONSES:
        if symptom in symptoms_lower:
            common_symptoms.append(symptom)

    if common_symptoms:
        # Use predefined responses for faster answers
        response = "Based on your symptoms, here's some general information:\n\n"
        for symptom in common_symptoms:
            response += f"**{symptom.capitalize()}**: {PREDEFINED_RESPONSES[symptom]}\n\n"

        response += "\n**IMPORTANT**: This is for informational purposes only. Please consult a healthcare professional for proper diagnosis and treatment."
        return response

    # For less common symptoms, use the model
    prompt = f"Based on these symptoms: {symptoms}. Provide brief possible conditions and general recommendations. Always emphasize consulting a doctor. Response should be concise and under 300 words."
    return generate_response_optimized(prompt, max_length=400)

def treatment_plan(condition: str, age: int, gender: str, medical_history: str) -> str:
    """Optimized treatment plan generation"""
    prompt = f"Generate a concise treatment plan for {condition} for a {age}-year-old {gender}. Medical history: {medical_history}. Include general guidelines and emphasize consulting a doctor. Keep response under 250 words."
    return generate_response_optimized(prompt, max_length=400)

# Professional CSS styling
css = """
:root {
    --primary: #2c7db1;
    --primary-dark: #1a5c8c;
    --secondary: #e8f4fc;
    --accent: #ff6b6b;
    --text: #333;
    --light-text: #666;
    --background: #f9fbfd;
    --card-bg: #ffffff;
    --border: #e0e6ed;
}

body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: var(--background);
    margin: 0;
    padding: 0;
    color: var(--text);
}

.gradio-container {
    background: var(--background) !important;
    max-width: 1200px !important;
    margin: 0 auto !important;
}

.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-dark) 100%);
    color: white;
    border-radius: 10px 10px 0 0;
    margin-bottom: 20px;
}

.header h1 {
    margin: 0;
    font-size: 2.5rem;
    font-weight: 600;
}

.header p {
    margin: 10px 0 0;
    opacity: 0.9;
}

.disclaimer {
    background-color: #fff4f4;
    border-left: 4px solid var(--accent);
    padding: 15px;
    margin: 15px 0;
    border-radius: 4px;
    font-size: 0.9rem;
}

.tab-nav {
    background: var(--card-bg) !important;
    border-radius: 8px !important;
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05) !important;
    padding: 5px !important;
}

.tab-nav button {
    border-radius: 6px !important;
    padding: 12px 24px !important;
    font-weight: 500 !important;
}

.tab-nav button.selected {
    background: var(--primary) !important;
    color: white !important;
}

.tab-content {
    background: var(--card-bg) !important;
    border: 1px solid var(--border) !important;
    border-radius: 10px !important;
    padding: 25px !important;
    margin-top: 15px !important;
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05) !important;
}

.input-panel, .output-panel {
    background: var(--secondary) !important;
    border-radius: 8px !important;
    padding: 20px !important;
    border: 1px solid var(--border) !important;
}

.input-panel h3, .output-panel h3 {
    margin-top: 0;
    color: var(--primary-dark);
    font-size: 1.2rem;
    border-bottom: 2px solid var(--primary);
    padding-bottom: 10px;
}

.gradio-button {
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-dark) 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 6px !important;
    padding: 12px 28px !important;
    font-weight: 500 !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
}

.gradio-button:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.15) !important;
}

.gradio-textbox, .gradio-dropdown, .gradio-number {
    border: 1px solid var(--border) !important;
    border-radius: 6px !important;
    padding: 12px !important;
    font-size: 16px !important;
    background: white !important;
}

.gradio-textbox:focus, .gradio-dropdown:focus, .gradio-number:focus {
    border-color: var(--primary) !important;
    box-shadow: 0 0 0 2px rgba(44, 125, 177, 0.2) !important;
}

.gradio-label {
    font-weight: 600 !important;
    margin-bottom: 8px !important;
    color: var(--primary-dark) !important;
}

.output-text {
    background: white !important;
    border: 1px solid var(--border) !important;
    border-radius: 8px !important;
    padding: 20px !important;
    min-height: 300px !important;
    line-height: 1.6 !important;
    overflow-y: auto !important;
}

.footer {
    text-align: center;
    margin-top: 30px;
    padding: 15px;
    color: var(--light-text);
    font-size: 0.85rem;
    border-top: 1px solid var(--border);
}

.loading {
    display: inline-block;
    width: 20px;
    height: 20px;
    border: 3px solid rgba(255,255,255,.3);
    border-radius: 50%;
    border-top-color: #fff;
    animation: spin 1s ease-in-out infinite;
}

@keyframes spin {
    to { transform: rotate(360deg); }
}
"""

# Create the enhanced Gradio interface
with gr.Blocks(css=css, theme=gr.themes.Default()) as app:
    with gr.Column(elem_classes="header"):
        gr.Markdown("# Medical AI Assistant")
        gr.Markdown("Advanced symptom analysis and treatment planning")

    with gr.Row():
        gr.Markdown("""
        <div class="disclaimer">
            <strong>Disclaimer:</strong> This AI assistant provides informational suggestions only and is not a substitute for professional medical advice. Always consult healthcare professionals for medical diagnosis and treatment.
        </div>
        """)

    with gr.Tabs(elem_classes="tab-nav") as tabs:
        with gr.TabItem("Disease Prediction", elem_classes="tab-content"):
            with gr.Row():
                with gr.Column(scale=1, elem_classes="input-panel"):
                    gr.Markdown("### Enter Patient Symptoms")
                    symptoms_input = gr.Textbox(
                        label="Symptoms",
                        placeholder="e.g., fever, headache, cough, fatigue...",
                        lines=3
                    )
                    predict_btn = gr.Button("Analyze Symptoms", elem_classes="gradio-button")

                with gr.Column(scale=1, elem_classes="output-panel"):
                    gr.Markdown("### Possible Conditions & Recommendations")
                    prediction_output = gr.Textbox(
                        label="",
                        lines=15,
                        elem_classes="output-text"
                    )

            predict_btn.click(
                disease_prediction,
                inputs=symptoms_input,
                outputs=prediction_output
            )

        with gr.TabItem("Treatment Plans", elem_classes="tab-content"):
            with gr.Row():
                with gr.Column(scale=1, elem_classes="input-panel"):
                    gr.Markdown("### Patient Information")
                    condition_input = gr.Textbox(
                        label="Medical Condition",
                        placeholder="e.g., diabetes, hypertension, migraine...",
                        lines=2
                    )
                    with gr.Row():
                        age_input = gr.Number(label="Age", value=30, minimum=1, maximum=120)
                        gender_input = gr.Dropdown(
                            choices=["Male", "Female", "Other"],
                            label="Gender",
                            value="Male"
                        )
                    history_input = gr.Textbox(
                        label="Medical History",
                        placeholder="Previous conditions, allergies, medications or None",
                        lines=2
                    )
                    plan_btn = gr.Button("Generate Treatment Plan", elem_classes="gradio-button")

                with gr.Column(scale=1, elem_classes="output-panel"):
                    gr.Markdown("### Personalized Treatment Plan")
                    plan_output = gr.Textbox(
                        label="",
                        lines=15,
                        elem_classes="output-text"
                    )

            plan_btn.click(
                treatment_plan,
                inputs=[condition_input, age_input, gender_input, history_input],
                outputs=plan_output
            )

    with gr.Row():
        gr.Markdown("""
        <div class="footer">
            Medical AI Assistant v1.0 | For informational purposes only | Consult a healthcare professional for medical advice
        </div>
        """)

# Launch the application
if __name__ == "__main__":
    app.launch(share=True, debug=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ff8b90b68253da536.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
